In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import Image
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import json
import numpy as onp
import jax
import jax.numpy as jnp
import flax
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import timecast as tc

from mpl_toolkits import mplot3d


plt.rcParams['figure.figsize'] = [20, 10]

import tqdm.notebook as tqdm

/home/dsuo/miniconda3/envs/skgaip/lib/python3.7/site-packages/jax/lib/xla_bridge.py:123: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [14]:
from timecast.utils.ar import _fit_constraint, compute_gram, fit_gram, historify
from timecast.utils import random

In [176]:
n = 15
input_dim = 4
output_dim = 2
history_len = 3

X = jax.random.uniform(random.generate_key(), shape=(n, input_dim))
Y = jax.random.uniform(random.generate_key(), shape=(n, output_dim))

XTX, XTY = compute_gram([(X, Y, None)], input_dim, output_dim, history_len)

In [ ]:
XTX_matrix = XTX.matrix(fit_intercept=True, input_dim)

In [183]:
kernel, bias = fit_gram(XTX, XTY)

In [184]:
kernel

DeviceArray([[[-0.12112668, -0.01588155],
              [ 0.29461107, -0.11640345],
              [-0.18669236,  0.17323935],
              [ 0.02627132, -0.05914355],
              [ 0.08157143,  0.09819189],
              [ 0.13814986,  0.04696539],
              [ 0.00360064, -0.08458078],
              [ 0.19632673, -0.08705389],
              [ 0.22611143, -0.13234228],
              [-0.09685083,  0.13079412],
              [-0.07473218, -0.0595607 ],
              [-0.06446776,  0.20944849]]], dtype=float32)

In [143]:
X_stats = XTX._feature_stats
Y_stats = XTX._output_stats

In [144]:
num_rc = input_dim

In [145]:
sumcol = jnp.tile(X_stats.sum, (num_rc, 1))

In [146]:
sumcol

DeviceArray([[5.2810802, 7.087426 , 6.8720837, 6.804742 , 5.4718595,
              6.239929 , 7.154348 , 6.7237587, 5.0525355, 6.825943 ,
              6.7483277, 6.985718 ],
             [5.2810802, 7.087426 , 6.8720837, 6.804742 , 5.4718595,
              6.239929 , 7.154348 , 6.7237587, 5.0525355, 6.825943 ,
              6.7483277, 6.985718 ],
             [5.2810802, 7.087426 , 6.8720837, 6.804742 , 5.4718595,
              6.239929 , 7.154348 , 6.7237587, 5.0525355, 6.825943 ,
              6.7483277, 6.985718 ],
             [5.2810802, 7.087426 , 6.8720837, 6.804742 , 5.4718595,
              6.239929 , 7.154348 , 6.7237587, 5.0525355, 6.825943 ,
              6.7483277, 6.985718 ]], dtype=float32)

In [147]:
result = jnp.concatenate((jnp.zeros((num_rc, XTX._matrix.shape[0])), XTX._matrix), axis=0)

In [148]:
result.shape

(16, 12)

In [149]:
result = jnp.concatenate((jnp.zeros((result.shape[0], num_rc)), result), axis=1)

In [150]:
result.shape

(16, 16)

In [151]:
result = jax.ops.index_update(result, jax.ops.index[:num_rc, :num_rc], X_stats.observations)

In [152]:
result

DeviceArray([[13.       , 13.       , 13.       , 13.       ,  0.       ,
               0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
               0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
               0.       ],
             [13.       , 13.       , 13.       , 13.       ,  0.       ,
               0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
               0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
               0.       ],
             [13.       , 13.       , 13.       , 13.       ,  0.       ,
               0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
               0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
               0.       ],
             [13.       , 13.       , 13.       , 13.       ,  0.       ,
               0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
               0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
               0.       ],
    

In [153]:
result = jax.ops.index_update(result, jax.ops.index[:num_rc, num_rc:], sumcol)

In [154]:
result

DeviceArray([[13.       , 13.       , 13.       , 13.       ,  5.2810802,
               7.087426 ,  6.8720837,  6.804742 ,  5.4718595,  6.239929 ,
               7.154348 ,  6.7237587,  5.0525355,  6.825943 ,  6.7483277,
               6.985718 ],
             [13.       , 13.       , 13.       , 13.       ,  5.2810802,
               7.087426 ,  6.8720837,  6.804742 ,  5.4718595,  6.239929 ,
               7.154348 ,  6.7237587,  5.0525355,  6.825943 ,  6.7483277,
               6.985718 ],
             [13.       , 13.       , 13.       , 13.       ,  5.2810802,
               7.087426 ,  6.8720837,  6.804742 ,  5.4718595,  6.239929 ,
               7.154348 ,  6.7237587,  5.0525355,  6.825943 ,  6.7483277,
               6.985718 ],
             [13.       , 13.       , 13.       , 13.       ,  5.2810802,
               7.087426 ,  6.8720837,  6.804742 ,  5.4718595,  6.239929 ,
               7.154348 ,  6.7237587,  5.0525355,  6.825943 ,  6.7483277,
               6.985718 ],
    

In [155]:
result = jax.ops.index_update(result, jax.ops.index[num_rc:, :num_rc], sumcol.T)

In [156]:
result

DeviceArray([[13.       , 13.       , 13.       , 13.       ,  5.2810802,
               7.087426 ,  6.8720837,  6.804742 ,  5.4718595,  6.239929 ,
               7.154348 ,  6.7237587,  5.0525355,  6.825943 ,  6.7483277,
               6.985718 ],
             [13.       , 13.       , 13.       , 13.       ,  5.2810802,
               7.087426 ,  6.8720837,  6.804742 ,  5.4718595,  6.239929 ,
               7.154348 ,  6.7237587,  5.0525355,  6.825943 ,  6.7483277,
               6.985718 ],
             [13.       , 13.       , 13.       , 13.       ,  5.2810802,
               7.087426 ,  6.8720837,  6.804742 ,  5.4718595,  6.239929 ,
               7.154348 ,  6.7237587,  5.0525355,  6.825943 ,  6.7483277,
               6.985718 ],
             [13.       , 13.       , 13.       , 13.       ,  5.2810802,
               7.087426 ,  6.8720837,  6.804742 ,  5.4718595,  6.239929 ,
               7.154348 ,  6.7237587,  5.0525355,  6.825943 ,  6.7483277,
               6.985718 ],
    

In [157]:
XTX_matrix = result

In [159]:
XTY_matrix = XTY.matrix(fit_intercept=True, input_dim=input_dim)

In [160]:
reg = 1.0 * jnp.eye(XTX_matrix.shape[0])
reg = jax.ops.index_update(reg, [0, 0], 0)
inv = onp.linalg.inv(XTX_matrix + reg)

In [161]:
inv

array([[ 5.22434950e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00, -3.69410872e-01, -4.50892061e-01,
        -4.02915478e-01, -4.22622204e-01, -1.68524697e-01,
        -3.99164528e-01, -3.93921345e-01, -2.25337401e-01,
        -2.03485832e-01, -6.53600872e-01, -1.10821731e-01,
        -4.49717551e-01],
       [-1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-1.00000000e+00,  0.00000000e

In [170]:
kernel, bias = fit_gram(XTX, XTY, input_dim=input_dim)

In [171]:
kernel

DeviceArray([[ 0.12047938,  0.03906086],
             [ 0.09251006,  0.02308223],
             [-0.06796371,  0.04429616]], dtype=float32)

In [172]:
bias

DeviceArray([0.07392025, 0.11869931], dtype=float32)

In [129]:
from timecast.utils.ar import _fit_constrained
beta = _fit_constrained(beta, inv, R, r)

In [130]:
beta

DeviceArray([[ 0.24008504],
             [ 0.24008505],
             [ 0.13840254],
             [ 0.13840254],
             [-0.10183237],
             [-0.10183237],
             [-0.12447697],
             [-0.12447697]], dtype=float32)

In [132]:
beta.take(jnp.arange(0, len(beta), input_dim), axis=0)

DeviceArray([[ 0.24008504],
             [ 0.13840254],
             [-0.10183237],
             [-0.12447697]], dtype=float32)